In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_Odil/image_indexing.csv')

In [ ]:
colonnes = ['id','files','date_created','timel_nature_place_term','timel_subject_term','timel_object_architecture_term', 'timel_character_term','timel_thema_term']

In [ ]:
df = df[colonnes]
df[['timel_nature_place_term','timel_subject_term', 'timel_object_architecture_term', 'timel_character_term', 'timel_thema_term']]=df[['timel_nature_place_term', 'timel_subject_term', 'timel_object_architecture_term', 'timel_character_term', 'timel_thema_term']].applymap(df_split_text)

In [ ]:
df=df[df['timel_subject_term'].notna()]

In [ ]:
path_dir = '/content/drive/MyDrive/'
df['files']=df['files'].apply(lambda file : os.path.join(path_dir, file)) #ajout chemin abs pour acces aux images

# Image captioning avec Florence-2

## Initialisation

In [ ]:
!pip install transformers==4.51.1

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modèle Florence-2-base
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/florence-2-base", trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained(
    "microsoft/florence-2-base", trust_remote_code=True)

# Modèle finetuned
#model=AutoModelForCausalLM.from_pretrained("francipaolo/florence-2-pal-comp-v2", trust_remote_code=True).to(device)
#processor = AutoProcessor.from_pretrained("francipaolo/florence-2-pal-comp-v2", trust_remote_code=True)


In [ ]:

class ImagePathDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        image = Image.open(path).convert("RGB")
        return image, path

    def __len__(self):
        return len(self.image_paths)

In [ ]:

def generate_batch(images, task_prompt="<DETAILED_CAPTION>", text_input=None):
    if text_input is None:
        prompts = [task_prompt] * len(images)
    elif isinstance(text_input, str):
        prompts = [task_prompt + text_input] * len(images)
    else:
        prompts = [task_prompt + t for t in text_input]

    inputs = processor(text=prompts, images=images, return_tensors="pt").to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            do_sample=False,
            num_beams=3,
        )

    generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
    image_sizes = [(img.width, img.height) for img in images]

    parsed_answers = [
        processor.post_process_generation(text, task=task_prompt, image_size=size)
        for text, size in zip(generated_texts, image_sizes)
    ]

    return parsed_answers, generated_ids


In [ ]:

#    Dataloader
image_paths = df['files'].tolist()
dataset = ImagePathDataset(image_paths)

def pil_collate(batch):
    images, paths = zip(*batch)
    return list(images), list(paths)

dataloader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=2, collate_fn=pil_collate)


In [ ]:
from tqdm import tqdm
# Inference
task_prompt = "<DETAILED_CAPTION>"
output_path = "/content/drive/MyDrive/data_Odil/captions_detailed_f-2base.jsonl"

with open(output_path, "a", encoding="utf-8") as f:
    for batch_images, batch_paths in tqdm(dataloader, desc="Generating captions"):
        parsed_answers, _ = generate_batch(
            task_prompt=task_prompt,
            images=batch_images
        )

        for path, answer in zip(batch_paths, parsed_answers):
            json.dump({path: answer}, f, ensure_ascii=False)
            f.write("\n")



